# Test McpExecutor Run Method

This notebook tests the `run` method of the `McpExecutor` class from `autocoder.common.mcp_tools`.

In [1]:
import os
import sys
import json
import asyncio
import tempfile
from pathlib import Path

from autocoder.common.mcp_hub import McpHub, McpServer, McpTool, McpResource, McpResourceTemplate
from autocoder.common.mcp_tools import McpExecutor
import byzerllm

In [2]:
# Helper function for async tests
async def run_test(test_func):
    """Helper to run async test functions"""
    try:
        await test_func()
        print("✅ Test passed")
    except AssertionError as e:
        print(f"❌ Test failed: {str(e)}")
    except Exception as e:
        print(f"❌ Test error: {str(e)}")

In [4]:
async def test_mcp_executor_run():
    """Test McpExecutor run method"""
    # Create settings file with filesystem server config
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        settings = {
            "mcpServers": {
                "filesystem": {
                    "command": "npx",
                    "args": [
                        "-y",
                        "@modelcontextprotocol/server-filesystem",
                        "/Users/allwefantasy/projects/tests"                        
                    ]
                }
            }
        }
        json.dump(settings, f)
        settings_path = f.name
    
    try:
        # Initialize hub and executor
        hub = McpHub(settings_path)
        await hub.initialize()
        
        llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
        executor = McpExecutor(hub, llm)
        
        # Create test conversation
        conversations = [{
            "role": "user",
            "content": "List the contents of /Users/allwefantasy/projects/tests"
        }]
        
        # Run the executor
        result = await executor.run(conversations)
        
        # Verify the result contains the directory listing
        assert len(result) > 2, "Expected result to contain directory listing"
        print("Test result:", result[-2]['content'])
        
    finally:
        # Cleanup
        await hub.shutdown()
        os.unlink(settings_path)

In [5]:
# Run the test
print("Testing McpExecutor run method:")
await run_test(test_mcp_executor_run)

Testing McpExecutor run method:


2025-01-02 18:29:53.002 | ERROR    | autocoder.common.mcp_hub:_fetch_resources:235 - Failed to fetch resources for filesystem: code=-32601 message='Method not found' data=None
2025-01-02 18:29:53.003 | ERROR    | autocoder.common.mcp_hub:_fetch_resource_templates:253 - Failed to fetch resource templates for filesystem: 'ClientSession' object has no attribute 'list_resource_templates'
2025-01-02 18:29:53.004 | INFO     | autocoder.common.mcp_hub:update_server_connections:194 - Connected to new MCP server: filesystem
2025-01-02 18:29:53.044 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2025-01-02 18:29:53,076	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2025-01-02 18:29:53,093	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


❌ Test error: 'coroutine' object is not iterable


/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/ipykernel_24056/1944525554.py:10: RuntimeWarning: coroutine 'McpExecutor.extract_mcp_calls' was never awaited
  print(f"❌ Test error: {str(e)}")
